In [1]:
import pandas as pd
from neo4j import GraphDatabase
import requests
from tqdm import tqdm
import itertools

In [2]:
tqdm.pandas()

In [3]:
driver = GraphDatabase.driver(uri="bolt://localhost:7687", auth=('neo4j', 'neo4j'))

# BISOGNA FARE UNA COSA 4 STAGE:
<ol>
    <li>usare i nomi degli autori presi dall'url dell'immagine e vedere di recuperare quelli non presenti in artgraph;</li>
    <li>ci saranno ancora altri lavori che non vengono presi. Proviamo a prendere il nome dell'autore da <code>artwork_info</code> e di rifare l'operazione
    <li>ci saranno ancora altri lavori che non vengono presi. Se l'autore riscontra delle opere via API, vuol dire che bisogna passare a prendere le info puntuali dell'artwork, quindi prendere il campo <code>url</code> e, infine, provare il matching;
    <li>in ultimo, se l'autore non dà alcun risultato, è molto probabile che ci sia un <b>typo</b> . Va riconosciuto, modificato il tutto e rilanciato
</ol>

Get artworks information

In [4]:
artwork_info = pd.read_csv('WikiArt-Emotions/WikiArt-info.tsv', delimiter='\t')

In [5]:
artwork_info['Artist Info URL'] = artwork_info['Artist Info URL'].progress_apply(
    lambda x: x if 'andrea-del-verrocchio' not in x \
        else '/'.join(x.split('/')[:-1] + ['andrea-del-verrochio']))
artwork_info

100%|██████████| 4119/4119 [00:00<00:00, 263571.76it/s]


,ID,Category,Artist,Title,Year,Image URL,Painting Info URL,Artist Info URL
0,58c6237dedc2c9c7dc0de1ae,Impressionism,Charles Courtney Curran,In the Luxembourg Garden,1889,https://uploads3.wikiart.org/00123/images/char...,https://www.wikiart.org/en/charles-courtney-cu...,https://www.wikiart.org/en/charles-courtney-cu...
1,577280dfedc2cb3880f28e76,Neo-Expressionism,Keith Haring,The Marriage of Heaven and Hell,1984,https://uploads1.wikiart.org/images/keith-hari...,https://www.wikiart.org/en/keith-haring/the-ma...,https://www.wikiart.org/en/keith-haring
2,57727f2dedc2cb3880ed5fa9,Post-Impressionism,Jozsef Rippl-Ronai,Uncle Piacsek in front of the Black Sideboard,1906,https://uploads3.wikiart.org/images/j-zsef-rip...,https://www.wikiart.org/en/jozsef-rippl-ronai/...,https://www.wikiart.org/en/jozsef-rippl-ronai
3,58d1240cedc2c94f900fc610,Cubism,Vadym Meller,Monk. For the Play &#39;Mazeppa&#39;,1920,https://uploads2.wikiart.org/00124/images/vady...,https://www.wikiart.org/en/vadym-meller/monk-f...,https://www.wikiart.org/en/vadym-meller
4,57727de7edc2cb3880e91f26,Romanticism,David Wilkie,The Defence of Sarago&#231;a,1828,https://uploads6.wikiart.org/images/david-wilk...,https://www.wikiart.org/en/david-wilkie/the-de...,https://www.wikiart.org/en/david-wilkie
...,...,...,...,...,...,...,...,...
4114,577287aeedc2cb388007fba9,Color Field Painting,Rupprecht Geiger,OE 260,1957,https://uploads7.wikiart.org/images/rupprecht-...,https://www.wikiart.org/en/rupprecht-geiger/oe...,https://www.wikiart.org/en/rupprecht-geiger
4115,57728001edc2cb3880efddcf,Surrealism,Oscar Dominguez,M&#225;quina de coser electro-sexual,1934,https://uploads4.wikiart.org/images/oscar-domi...,https://www.wikiart.org/en/oscar-dominguez/m-q...,https://www.wikiart.org/en/oscar-dominguez
4116,57728ac7edc2cb3880123cc2,Neo-Expressionism,Georg Baselitz,Female Nude on a Kitchen Chair,1979,https://uploads2.wikiart.org/images/georg-base...,https://www.wikiart.org/en/georg-baselitz/fema...,https://www.wikiart.org/en/georg-baselitz
4117,57728412edc2cb3880fc9ff4,Expressionism,Marie Laurencin,Apollinaire and His Friends,1909,https://uploads0.wikiart.org/images/marie-laur...,https://www.wikiart.org/en/marie-laurencin/apo...,https://www.wikiart.org/en/marie-laurencin


In [6]:
def mod_painting_info(link):
    l = link.split('/')
    l[-2] = 'andrea-del-verrochio'
    return '/'.join(l)

In [7]:
def mod_painting_url(link):
    l = link.split('/')
    l[-2] = 'andrea-del-verrochio'
    return '/'.join(l)

In [8]:
artwork_info['Painting Info URL'] = artwork_info['Painting Info URL'].progress_apply(
    lambda x: x if 'andrea-del-verrocchio' not in x \
        else mod_painting_info(x))
artwork_info['Image URL'] = artwork_info['Image URL'].progress_apply(
    lambda x: x if 'andrea-del-verrocchio' not in x \
        else mod_painting_info(x))
artwork_info

100%|██████████| 4119/4119 [00:00<?, ?it/s]


,ID,Category,Artist,Title,Year,Image URL,Painting Info URL,Artist Info URL
0,58c6237dedc2c9c7dc0de1ae,Impressionism,Charles Courtney Curran,In the Luxembourg Garden,1889,https://uploads3.wikiart.org/00123/images/char...,https://www.wikiart.org/en/charles-courtney-cu...,https://www.wikiart.org/en/charles-courtney-cu...
1,577280dfedc2cb3880f28e76,Neo-Expressionism,Keith Haring,The Marriage of Heaven and Hell,1984,https://uploads1.wikiart.org/images/keith-hari...,https://www.wikiart.org/en/keith-haring/the-ma...,https://www.wikiart.org/en/keith-haring
2,57727f2dedc2cb3880ed5fa9,Post-Impressionism,Jozsef Rippl-Ronai,Uncle Piacsek in front of the Black Sideboard,1906,https://uploads3.wikiart.org/images/j-zsef-rip...,https://www.wikiart.org/en/jozsef-rippl-ronai/...,https://www.wikiart.org/en/jozsef-rippl-ronai
3,58d1240cedc2c94f900fc610,Cubism,Vadym Meller,Monk. For the Play &#39;Mazeppa&#39;,1920,https://uploads2.wikiart.org/00124/images/vady...,https://www.wikiart.org/en/vadym-meller/monk-f...,https://www.wikiart.org/en/vadym-meller
4,57727de7edc2cb3880e91f26,Romanticism,David Wilkie,The Defence of Sarago&#231;a,1828,https://uploads6.wikiart.org/images/david-wilk...,https://www.wikiart.org/en/david-wilkie/the-de...,https://www.wikiart.org/en/david-wilkie
...,...,...,...,...,...,...,...,...
4114,577287aeedc2cb388007fba9,Color Field Painting,Rupprecht Geiger,OE 260,1957,https://uploads7.wikiart.org/images/rupprecht-...,https://www.wikiart.org/en/rupprecht-geiger/oe...,https://www.wikiart.org/en/rupprecht-geiger
4115,57728001edc2cb3880efddcf,Surrealism,Oscar Dominguez,M&#225;quina de coser electro-sexual,1934,https://uploads4.wikiart.org/images/oscar-domi...,https://www.wikiart.org/en/oscar-dominguez/m-q...,https://www.wikiart.org/en/oscar-dominguez
4116,57728ac7edc2cb3880123cc2,Neo-Expressionism,Georg Baselitz,Female Nude on a Kitchen Chair,1979,https://uploads2.wikiart.org/images/georg-base...,https://www.wikiart.org/en/georg-baselitz/fema...,https://www.wikiart.org/en/georg-baselitz
4117,57728412edc2cb3880fc9ff4,Expressionism,Marie Laurencin,Apollinaire and His Friends,1909,https://uploads0.wikiart.org/images/marie-laur...,https://www.wikiart.org/en/marie-laurencin/apo...,https://www.wikiart.org/en/marie-laurencin


Get name for trying to do a mapping with ArtGraph

In [9]:
artwork_info['name'] = artwork_info['Image URL'].map(lambda x: '_'.join(x.split('/')[-2:]))

Function that checks if a painting is present in ArtGraph

In [10]:
def is_in_artgraph(param, value, driver):
    base_query = f'match (a:Artwork) where a.{param} = "{value}" return count(distinct a) as num'
    with driver.session(database='neo4j') as session:
        ans = session.run(base_query).data()
    return ans[0]['num']

In [11]:
artwork_info['name_in_artgraph'] = artwork_info.progress_apply(lambda x: is_in_artgraph('name', x['name'], driver), axis=1)

100%|██████████| 4119/4119 [09:05<00:00,  7.55it/s]


In [12]:
artwork_info['artist'] = artwork_info['name'].map(lambda x: x.split('_')[0])
artwork_info['artist_1'] = artwork_info['Artist'].map(lambda x: '-'.join(x.lower().split(' ')))

In [13]:
artwork_info['name'] = artwork_info['name'].map(lambda x: x.split('_')[1])

Taking artworks that are not present in artgraph

In [14]:
unretrieved = artwork_info[artwork_info.name_in_artgraph == 0]

In [15]:
unretrieved.drop(['Category', 'Artist', 'Year', 'Painting Info URL', 'Artist Info URL', 'name_in_artgraph','Image URL'],
                 axis = 1, inplace=True)

C:\Users\enzos.LAPTOP-RB20LOIL\AppData\Local\Temp\ipykernel_43660\107577158.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unretrieved.drop(['Category', 'Artist', 'Year', 'Painting Info URL', 'Artist Info URL', 'name_in_artgraph','Image URL'],


In [16]:
# function that gets all the paintings of a specific artist
def get_paintings_by_artist(artist_name):
    base_query = 'https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl={artist}&json=2'
    return requests.get(base_query.format(artist=artist_name)).json()

# function that gets all painting infos for a specific artwork
def get_painting_info(painting_id: int):
    base_query = 'https://www.wikiart.org/en/App/Painting/ImageJson/{painting}'
    return requests.get(base_query.format(painting=painting_id)).json()

# function that gets all url unmatched artworks with respect to wikiart artwork info
def retrieve_artworks_by_url(artist_name, metadata):
    paintings = get_paintings_by_artist(artist_name)
    paintings = map(lambda x: x['contentId'], paintings)
    paintings = list(map(lambda x: f"{get_painting_info(x)['url']}.jpg", paintings))
    return metadata[metadata['name'].isin(paintings)].name.tolist()

# function that gets all unmatched artworks with respect to basic info got by searching just paintings of spceific artwork
def retrieve_artworks(artist_name, metadata):
    paintings = get_paintings_by_artist(artist_name)
    titles = list(map(lambda x: x['title'], paintings))
    names = list(map(lambda x: x['image'].split('/')[-1][:-10], paintings))
    return metadata[(metadata['Title'].isin(titles)) | (metadata['name'].isin(names))].name.tolist()

Group paintings by artist

In [17]:
grouped = unretrieved.groupby(['artist', 'artist_1'])[['Title', 'name']].apply(lambda x: x.values.tolist()).reset_index(name='metadata')

Retrieve artworks following **1**.

In [17]:
retrieved = grouped.progress_apply(lambda x: retrieve_artworks(x.artist, pd.DataFrame(x.metadata, columns=['Title', 'name'])), axis=1)

  5%|▍         | 23/501 [00:07<02:28,  3.21it/s]


In [ ]:
total = unretrieved.groupby(['artist', 'artist_1'])['name'].apply(list).reset_index(name='names')

placing the first retrieved artwroks

In [ ]:
total['ret'] = retrieved

Making comparison and underline unretrieved artworks

In [ ]:
total['difference'] = total.apply(lambda x: list(set(x.names) - set(x.ret)), axis=1)
total

In [ ]:
total.difference.map(len).sum()

In [ ]:
paintings_still_unretrieved = list(itertools.chain(*total.difference.values.tolist()))
artwork_info['api_v1_artist'] = artwork_info.progress_apply(lambda x: 1 if x['name'] not in paintings_still_unretrieved and not x.name_in_artgraph else 0, axis=1)

Attualmente ce ne mancano 51

# STAGE 2: USARE L'ALTRO NOME

In [ ]:
unretrieved = unretrieved[unretrieved.name.isin(paintings_still_unretrieved)]

In [ ]:
grouped = unretrieved.groupby(['artist', 'artist_1'])[['Title', 'name']].apply(lambda x: x.values.tolist()).reset_index(name='metadata')
retrieved = grouped.progress_apply(lambda x: retrieve_artworks(x.artist_1, pd.DataFrame(x.metadata, columns=['Title', 'name'])), axis=1)

In [ ]:
total = unretrieved.groupby(['artist', 'artist_1'])['name'].apply(list).reset_index(name='names')
total['ret'] = retrieved
total['difference'] = total.apply(lambda x: list(set(x.names) - set(x.ret)), axis=1)

In [ ]:
total

In [ ]:
total[total.difference.map(len) != 0]

# STAGE **3**
## RECURSIVE QUERY BASED TO URL

In [ ]:
paintings_still_unretrieved = list(itertools.chain(*total.difference.values.tolist()))


In [ ]:
artwork_info['api_v1_artist_1'] = artwork_info.progress_apply(lambda x: 1 if x['name'] not in paintings_still_unretrieved and not x.api_v1_artist else 0, axis=1)

In [ ]:
unretrieved = unretrieved[unretrieved.name.isin(paintings_still_unretrieved)]

In [ ]:
grouped = unretrieved.groupby(['artist', 'artist_1'])[['Title', 'name']].apply(lambda x: x.values.tolist()).reset_index(name='metadata')
retrieved = grouped.progress_apply(lambda x: retrieve_artworks_by_url(x.artist, pd.DataFrame(x.metadata, columns=['Title', 'name'])), axis=1)

In [ ]:
total = unretrieved.groupby(['artist', 'artist_1'])['name'].apply(list).reset_index(name='names')
total['ret'] = retrieved
total['difference'] = total.apply(lambda x: list(set(x.names) - set(x.ret)), axis=1)

In [ ]:
total[total.difference.map(len) != 0]

# STAGE 4

In [ ]:
paintings_still_unretrieved = list(itertools.chain(*total.difference.values.tolist()))
artwork_info['api_v1_url'] = artwork_info.progress_apply(lambda x: 1 if x['name'] not in paintings_still_unretrieved and not x.api_v1_artist_1 else 0, axis=1)

In [ ]:
unretrieved = unretrieved[unretrieved.name.isin(paintings_still_unretrieved)]

In [ ]:
def get_painting(id: str):
    base = 'https://www.wikiart.org/en/api/2/Painting?id={id}'
    return requests.get(base.format(id=id)).json()

In [ ]:
unretrieved['api_v2'] = unretrieved.ID.progress_apply(lambda x: 'title' in get_painting(x))
unretrieved

In [ ]:
artwork_info['api_v2'] = artwork_info.progress_apply(lambda x: 1 if x['name'] not in unretrieved[unretrieved.api_v2==False]['name'].tolist() and not x.api_v1_url else 0, axis=1)

In [ ]:
unretrieved[unretrieved.api_v2==False]

In [ ]:
unretrieved[unretrieved.api_v2==False].to_csv('still_unretrieved.csv')

In [ ]:
artwork_info

In [ ]:
artwork_info.to_csv('artwork_info_sources.csv')